#Paquetes necesarios

In [2]:
import cv2  
import math 

from ultralytics import YOLO
import easyocr
import pytesseract
import matplotlib.pyplot as plt
import imutils
import numpy as np



In [3]:
# Función para leer matrículas usando easyOCR y procesamiento de imágenes
def readLicensePlate(image):

    # Convertir a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar un filtro Gaussiano para suavizar la imagen y reducir el ruido
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Aplicar umbralización
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Crear un objeto Reader de EasyOCR
    reader = easyocr.Reader(['en'])

    # Obtener las regiones de texto utilizando EasyOCR
    results = reader.readtext(thresh)

    print(results)

    return results

    # Mostrar la imagen con los resultados
    # cv2.imshow('OCR Result', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # if len(result) > 0: 
    #     print("Resultado: " + result[0][1])
    #     return result[0][2]
    # else: 
    #     print("No se ha encontrado texto.")
    #     return None
# img = cv2.imread('images/images/Cars3.png')
# readLicensePlate(img)


In [4]:
# Función para determinar la ubicación de la matrícula
def searchLicensePlate(image, coords):
    subImg = image[coords[2]:coords[3], coords[0]:coords[1]]
    subImg = resizeImage(subImg, 0.2, 0.1, 0.4, 0.2)
    return subImg

# Función para redimensionar una imagen porcentualmente por cada uno de sus 4 lados
def resizeImage(image, leftToRight, rightToLeft, topToBottom, bottomToTop):
    sizes = image.shape
    subImg = image[round(sizes[0]*topToBottom):sizes[0]-round(sizes[0]*bottomToTop),round(sizes[1]*leftToRight):sizes[1]-round(sizes[1]*rightToLeft)]

    return subImg

# Función para usar un modelo de YOLO
def useModel(pathModel, img, license):
    model = YOLO(pathModel)
    resultsModel = model(img, stream=True)
    boxesFiltered = []
    for r in resultsModel:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            if not license:
                if cls != 2:
                    continue
            boxesFiltered.append(box)
    return boxesFiltered

# Función para mostrar los resultados obtenidos de un modelo YOLO
def printResultsInImage(boxes, img):
    i = 0
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                
        # Confianza
        confidence = math.ceil((box.conf[0]*100))/100
        # print("Confidence --->",confidence)

        # Clase
        cls = int(box.cls[0])

        if cls == 2:
            # print("Class name --> car")
            cv2.putText(img, 'car', [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        else:
            # print("Class name --> license")
            cv2.putText(img, 'license', [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)
            
        subImg = img[y1:y2,x1:x2]

        i += 1
        cv2.imwrite('images/result' + str(i) + '.jpg', subImg)

        # readLicensePlate(subImg)

    cv2.imwrite('images/result.jpg', img)
    cv2.imshow('images/result.jpg', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# img = cv2.imread('images/images/Cars104.png')
# boxes = useModel('yolov8n.pt', img, False)
# printResultsInImage(boxes, img)
# mixModels(img, classNamesOriginalModel)
# showResultsModel(results, classNamesOriginalModel, img)

In [11]:
# Función para mezclar modelos de YOLO, de detección de vehículos y de matrículas y así obtener resultados más precisos
def mixModels(img):
    boxesCars = useModel('yolov8n.pt', img, False)
    results = []
    imgcpy = img

    for box in boxesCars:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        x1_1, y1_1, x2_1, y2_1 = int(x1), int(y1), int(x2), int(y2)

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)

        subImg = img[y1:y2,x1:x2]
        # cv2.imshow('car', subImg)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        boxesLicenses = useModel('yolov8n-licenseTrained.pt', subImg, True)
        for boxL in boxesLicenses:
            x1, y1, x2, y2 = boxL.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
            licenseImg = subImg[y1:y2,x1:x2]
            results = readLicensePlate(licenseImg)
            if results:
                print(results[0][1])
                cv2.rectangle(subImg, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cv2.putText(subImg, results[0][1] + '(' + str(round(results[0][2]*100, 2)) + '%)', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            imgcpy[y1_1:y2_1, x1_1:x2_1] = subImg

            # cv2.imshow('license', imgcpy)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
        results.extend(boxesLicenses)
    
    return results
        
        

# img = cv2.imread('images/images/Cars7.png')
# boxes = mixModels(img)
# printResultsInImage(boxes, img)

In [14]:
# Función para pobar el modelo mixto en un vídeo
def videoTest(pathVideo, pathOutputVideo):
    cap = cv2.VideoCapture(pathVideo)

    if not cap.isOpened():
        print("Error al abrir el vídeo")    
        exit()

    width = int(cap.get(3))
    height = int(cap.get(4))
    fps = cap.get(5)

    fourcc = cv2.VideoWriter_fourcc(*'H264')
    out = cv2.VideoWriter(pathOutputVideo, fourcc, fps, (width, height))
    
    nFrames = 0
    while True:
        ret, frame = cap.read()

        if not ret:
            break
        
        nFrames += 1
        if nFrames % 100 == 0:
            # mixModels(frame)
            print(nFrames)
            # cv2.imshow('Video', frame)
            # key = cv2.waitKey(0) & 0xFF
            
        # mixModels(frame)
        # out.write(frame)
        
        # cv2.imshow('Video', frame)

        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
    
    cap.release()
    # out.release()
    cv2.destroyAllWindows()
    print(nFrames)


videoTest('videos/Traffic IP Camera.mp4', 'videos/test1.mp4')

100
200
300
400
500
600
700
749
